In [ ]:
# Colab에 Mecab 설치
!git clone https://github.com/SOMJANG/Mecab-ko-for-Google-Colab.git
%cd Mecab-ko-for-Google-Colab
!bash install_mecab-ko_on_colab190912.sh

Cloning into 'Mecab-ko-for-Google-Colab'...
remote: Enumerating objects: 91, done.
remote: Counting objects: 100% (91/91), done.
remote: Compressing objects: 100% (85/85), done.
remote: Total 91 (delta 43), reused 22 (delta 6), pack-reused 0
Unpacking objects: 100% (91/91), done.
/content/Mecab-ko-for-Google-Colab
Installing konlpy.....
     |████████████████████████████████| 19.4 MB 1.3 MB/s 
     |████████████████████████████████| 448 kB 49.2 MB/s 
     |████████████████████████████████| 86 kB 5.5 MB/s 
  Attempting uninstall: beautifulsoup4
    Found existing installation: beautifulsoup4 4.6.3
    Uninstalling beautifulsoup4-4.6.3:
      Successfully uninstalled beautifulsoup4-4.6.3
Done
Installing mecab-0.996-ko-0.9.2.tar.gz.....
from https://bitbucket.org/eunjeon/mecab-ko/downloads/mecab-0.996-ko-0.9.2.tar.gz
--2021-08-06 04:08:53--  https://bitbucket.org/eunjeon/mecab-ko/downloads/mecab-0.996-ko-0.9.2.tar.gz
Resolving bitbucket.org (bitbucket.org)... 104.192.141.1, 2406:da00:ff00

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import urllib.request
from collections import Counter
from konlpy.tag import Mecab
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.tokenize import RegexpTokenizer
import numpy as np
import matplotlib.pyplot as plt
import io

df=pd.read_csv('/content/drive/My Drive/your data',  error_bad_lines=False)

df.head(10)

,Unnamed: 0,Place,Address,Name,Review,Date,Score
0,0,전주 한옥 마을,-,diditall,전주가면 무조건 가봐야하는곳한옥하나하나 너무 멋지고 잘 꾸며져 있어요그러나 길거리 ...,2021년 5월 25일,4.0
1,1,전주 한옥 마을,-,hyun,"젊음의 거리, 사진찍는 거리입니다. 저렴하게 빌리는 옷 덕분에 여기 저기 한복 입고...",2021년 5월 6일,4.0
2,2,전주 한옥 마을,-,l*alalala,한복입고 사진찍긴 좋아요거리음식거리라서 먼가 특별하다 싶지 않아요물론 처음가서 보기...,2021년 5월 4일,3.0
3,3,전주 한옥 마을,-,ashurei2000,전주여행을 생각한다면 꼭 한번 가게되는 곳이라고 생각합니다.한옥을 보는 재미도 있지...,2020년 9월 18일,4.0
4,4,전주 한옥 마을,-,blessmj,첫째날은 사람구경둘째날은 먹거리 먹고 꽃구경 셋째날은 세세한 곳 소리관 김치 그...,2020년 5월 28일,5.0
5,5,전주 한옥 마을,-,사장님,전주 한옥마을을 평일에 갔더니 한산하니 좋았어요.길에 쓰레기 하나 없이 너무 깨끗했...,2020년 2월 4일,3.0
6,6,전주 한옥 마을,-,composuretour,명절 긴 이동거리를 고려하여 중간 휴식지로 전주 한옥마을을 선택명절 직전 한옥 마을...,2020년 1월 29일,3.0
7,7,전주 한옥 마을,-,Oh!,한옥마을 거리가 온통 가게들로 바뀌었네요. 사람이 북적거리는 건 좋은데 고즈넉한 맛...,2020년 1월 14일,3.0
8,8,전주 한옥 마을,-,panclass,"오랜만에 가봤는 데, 깨끗하게 관리 되고 있는 듯 보였고, 이쁜 한복체험하는 곳도 ...",2020년 1월 2일,5.0
9,9,전주 한옥 마을,-,ahirity,전주에 내려간 여행객들이 모두 모이는 곳이다. 먹자거리가 형성되어 있어서 길을 걷다...,2019년 12월 9일,4.0


In [ ]:
cdf=df[['Review']]
print(type(cdf))

<class 'pandas.core.frame.DataFrame'>


In [ ]:
drop_index = cdf[cdf['Review'] == '-'].index
cdf=cdf.drop(drop_index)
cdf.head()

,Review
0,전주가면 무조건 가봐야하는곳한옥하나하나 너무 멋지고 잘 꾸며져 있어요그러나 길거리 ...
1,"젊음의 거리, 사진찍는 거리입니다. 저렴하게 빌리는 옷 덕분에 여기 저기 한복 입고..."
2,한복입고 사진찍긴 좋아요거리음식거리라서 먼가 특별하다 싶지 않아요물론 처음가서 보기...
3,전주여행을 생각한다면 꼭 한번 가게되는 곳이라고 생각합니다.한옥을 보는 재미도 있지...
4,첫째날은 사람구경둘째날은 먹거리 먹고 꽃구경 셋째날은 세세한 곳 소리관 김치 그...


In [ ]:
cdf_l=list(np.array(cdf.Review.tolist()))

In [ ]:
import re
clean_text = []

for n in range(len(cdf_l)):
    
    try:
      line = cdf_l[n]
      l = re.compile('[^ ㄱ-ㅣ가-힣]+') # 한글을 뺀 나머지 l에 저장
      text = l.sub("",line) # l을 line에서 지움
      clean_text.append(text)
    except : pass
  
print(clean_text)

['전주가면 무조건 가봐야하는곳한옥하나하나 너무 멋지고 잘 꾸며져 있어요그러나 길거리 음식들이 너무 난잡하게 되어있어요간단하게 둘러보시고 사진찍기 좋아요', '젊음의 거리 사진찍는 거리입니다 저렴하게 빌리는 옷 덕분에 여기 저기 한복 입고 다니며 사진 찍는 사람들이 많읍니다 옷도 예쁘고 머리 장식까지 예쁘게 잘 해주는데 한 번쯤 해볼만 합니다 나이 드신 분들도 여고 동창들 끼리 모녀들도 와서 하느데 보기 좋더라구요  길거리 음식도 저렴하고 맛있읍니다', '한복입고 사진찍긴 좋아요거리음식거리라서 먼가 특별하다 싶지 않아요물론 처음가서 보기에는 좋고 골목골목돌아다니면    길이나 돌담이나 예쁘긴 해요하지만 거리음식거리같아요 어느순간 한옥마을은 사라지고 음식점만 남았엉ㄷ', '전주여행을 생각한다면 꼭 한번 가게되는 곳이라고 생각합니다한옥을 보는 재미도 있지만길거리 여러가지 음식들도 여행의 즐거움을 더해줍니다특히나 육전과 문어꼬치가 맛있었어요그리고 한국 전통의 얼레빗을 파는 매장을 발견했는데 무형문화재 이상근목소장님이하나 하나 만드시는 것으로 너무 아름답고 좋았습니다전주 마을을 가신다면 방문해보시고 선물용으로 장만하셔도 좋을거 같아요저도 가격이 좀 비쌌지만 마음에 드는 얼레빗을 장만했지요', '첫째날은 사람구경둘째날은 먹거리 먹고 꽃구경 셋째날은 세세한 곳 소리관 김치   그릇구경등  볼거리가 소소하게 많아 좋았어요', '전주 한옥마을을 평일에 갔더니 한산하니 좋았어요길에 쓰레기 하나 없이 너무 깨끗했고 주차장도 잘 되어 있어서 구경하기에 좋았답니다요즘엔 한복뿐만 아니라 경성의복 대여점도 있어 경성의복 대여해서 입고 다니는 분들도 많아서 잼있었어요', '명절 긴 이동거리를 고려하여 중간 휴식지로 전주 한옥마을을 선택명절 직전 한옥 마을 그리고 한옥 펜션을 구경하니 더욱 명절 스럽고나름 의미있는 기분 전환의 시간을 보낼 수 있음', '한옥마을 거리가 온통 가게들로 바뀌었네요 사람이 북적거리는 건 좋은데 고즈넉한 맛은 사라져서 아쉽기만 하네요 정체불명의 한복들도 많이 보

In [ ]:
!pip install git+https://github.com/ssut/py-hanspell.git

  Cloning https://github.com/ssut/py-hanspell.git to /tmp/pip-req-build-u32sy2t5
  Running command git clone -q https://github.com/ssut/py-hanspell.git /tmp/pip-req-build-u32sy2t5
  Created wheel for py-hanspell: filename=py_hanspell-1.1-py3-none-any.whl size=4870 sha256=75138630fc7abb6c24ee70d84f9859709d2af2bd9465c8cc7d20a97ff303d0d4
  Stored in directory: /tmp/pip-ephem-wheel-cache-6pkxzhe7/wheels/ab/f5/7b/d4124bb329c905301baed80e2ae45aa14e824f62ebc3ec2cc4
Successfully built py-hanspell


In [ ]:
from hanspell import spell_checker

sent = "맞춤법 틀리면 외 않되? 쓰고싶은대로쓰면돼지 "
spelled_sent = spell_checker.check(sent)
hanspell_sent = spelled_sent.checked


print(hanspell_sent)

맞춤법 틀리면 왜 안돼? 쓰고 싶은 대로 쓰면 되지


In [ ]:
hansp=[]
for i in clean_text:
  print(i)
  sent = i
  spelled_sent = spell_checker.check(sent)
  hanspell_sent = spelled_sent.checked
  hansp.append(hanspell_sent)

print(hansp)

In [ ]:
!pip install soynlp

     |████████████████████████████████| 416 kB 5.1 MB/s 


In [ ]:
import urllib.request
from soynlp import DoublespaceLineCorpus
from soynlp.word import WordExtractor
corpus = DoublespaceLineCorpus("2016-10-20.txt")

word_extractor = WordExtractor()
word_extractor.train(corpus)
word_score_table = word_extractor.extract()

from soynlp.tokenizer import MaxScoreTokenizer
scores = {word:score.cohesion_forward for word, score in word_score_table.items()}
maxscore_tokenizer = MaxScoreTokenizer(scores=scores)


local variable 'f' referenced before assignment
training was done. used memory 0.439 Gb
all cohesion probabilities was computed. # words = 0
all branching entropies was computed # words = 0
all accessor variety was computed # words = 0


In [ ]:
Soynlp=[]
for i in hansp:
  maxscore_tokenizer = MaxScoreTokenizer(scores=scores)
  m = maxscore_tokenizer.tokenize(i)
  Soynlp.append(m)

print(Soynlp)

[['전주', '가면', '무조건', '가봐야', '하는', '곳', '한옥', '하나하나', '너무', '멋지고', '잘', '꾸며져', '있어요', '그러나', '길거리', '음식들이', '너무', '난잡하게', '되어있어요', '간단하게', '둘러보시고', '사진', '찍기', '좋아요'], ['젊음의', '거리', '사진', '찍는', '거리입니다', '저렴하게', '빌리는', '옷', '덕분에', '여기저기', '한복', '입고', '다니며', '사진', '찍는', '사람들이', '많습니다', '옷도', '예쁘고', '머리', '장식까지', '예쁘게', '잘', '해주는데', '한', '번쯤', '해볼', '만합니다', '나이', '드신', '분들도', '여고', '동창들끼리', '모녀들도', '와서', '하는데', '보기', '좋더라고요', '길거리', '음식도', '저렴하고', '맛있습니다'], ['한복', '입고', '사진', '찍긴', '좋아요', '거리', '음식거리라서', '먼가', '특별하다', '싶지', '않아요', '물론', '처음', '가서', '보기에는', '좋고', '골목골목', '돌아다니면', '길이나', '돌담이나', '예쁘긴', '해요', '하지만', '거리', '음식거리', '같아요', '어느', '순간', '한옥마을은', '사라지고', '음식점만', '남았어ᄃ'], ['전주여행을', '생각한다면', '꼭', '한번', '가게', '되는', '곳이라고', '생각합니다', '한옥을', '보는', '재미도', '있지만', '길거리', '여러', '가지', '음식들도', '여행의', '즐거움을', '더해줍니다', '특히나', '육전과', '문어', '꼬치가', '맛있었어요', '그리고', '한국', '전통의', '얼레빗을', '파는', '매장을', '발견했는데', '무형문화재', '이상근', '목소장님이', '하나', '하나', '만드시는', '것으로', '너무', '아름답고', '좋았습니다', '전주', '마을을', '가신다면

In [ ]:
dic=pd.read_csv('/content/drive/My Drive/sundo soft/senti3.csv',  error_bad_lines=False)
dic.head()

,word,polarity
0,ㅡㅡ,-1
1,ㅠㅠ,-1
2,ㅠ_ㅠ,-1
3,ㅠ,-1
4,ㅜㅡ,-1


In [ ]:
dic_w=list(np.array(dic.word.tolist()))
print(dic_w[:10])

['ㅡㅡ', 'ㅠㅠ', 'ㅠ_ㅠ', 'ㅠ', 'ㅜㅡ', 'ㅜㅜ', 'ㅜ_ㅜ', 'ㅜ.ㅜ', 'ㅜ', 'ㅗ']


In [ ]:
dic_p=list(np.array(dic.polarity.tolist()))
print(dic_p[:10])

[-1, -1, -1, -1, -1, -1, -1, -1, -1, -1]


In [ ]:
s=[]
for i in range(len(Soynlp)):
  sent=0
  for j in range(len(Soynlp[i])):
    for k in range(len(dic_w)):
      if (Soynlp[i][j]==dic_w[k]):
        sent+=dic_p[k]
  s.append(sent)

print(s)

[3, 5, 3, 6, 0, 1, 0, 0, 3, 4, 1, 0, 0, 3, 4, 1, 2, -2, 0, 2, 1, 2, -2, 0, 2, 0, 0, 3, 1, 0, 0, 0, 3, 1, 0, -2, 5, 3, 0, 0, -2, 5, 3, 0, 0, 0, -3, -1, 0, 0, 0, -3, -1, 0, 0, 4, 4, 0, 3, 0, 4, 4, 0, 3, 0, 2, 0, -4, 0, -1, 2, 0, -4, 0, -1, 0, 5, 4, -1, 3, 0, 5, 4, -1, 3, 0, 3, 2, -1, 0, 0, 3, 2, -1, 0, 2, 3, 2, 1, 5, 2, 3, 2, 1, 5, 0, -2, 1, 7, 2, 0, -2, 1, 7, 2, 7, 0, 1, 4, 1, 7, 0, 1, 4, 1, 4, 2, 0, 0, 2, 4, 2, 0, 0, 2, 0, 0, 1, -2, 0, 0, 0, 1, -2, 0, 0, 2, 1, 2, 0, 0, 2, 1, 2, 0, 0, 2, 0, 0, 1, 0, 2, 0, 0, 1, 0, 0, 4, 0, 1, 0, 0, 4, 0, 1, -1, 0, 0, 3, 0, 0, 0, 0, 2, 3, 1, 1, 0, 6, 3, 1, 1, 0, 6, 3, 3, 3, 3, 1, 1, 3, 3, 3, 1, 1, -4, 0, 0, 0, 1, -4, 0, 0, 0, 1, 2, 0, 0, 1, 2, 2, 0, 0, 1, 2, 2, 2, 1, 1, 1, 2, 2, 1, 1, 1, -1, 2, 3, 2, 2, -1, 2, 3, 2, 2, 2, 2, 2, 0, 2, 1, 1, 0, 0, 0, 2, 0, 0, -1, 0, 0, -1, -2, 0, -1, 0, 1, -2, 0, 2, 6, 0, 2, 3, -1, 0, -1, 2, -2, 1, 3, 0, 0, 0, 2, 6, 1, 3, 0, 2, 0, 4, 0, 4, 0, 0, -1, 4, 0, 2, 2, 1, 1, 2, 2, 1, 5, 0, 0, 0, 0, 0, 2, 3, 2, 0, 2, 2, 1, 1, 1, 2,

In [ ]:
cdf=df[['Review', 'Score']]
cdf.Score=pd.to_numeric(cdf.Score, errors='coerce')
cdf=cdf.dropna()

/usr/local/lib/python3.7/dist-packages/pandas/core/generic.py:5170: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


In [ ]:
total_data=cdf
pd.to_numeric(total_data.Score)
df_negative = total_data[total_data.Score < 3].copy()
df_negative['label'] = 0
df_negative.head()
print(len(df_negative))

34


In [ ]:
df_positive = total_data[total_data.Score >= 3].copy()
df_positive['label'] = 1
df_positive.head()
print(len(df_positive))

725


In [ ]:
tdf = pd.concat([df_positive,df_negative])
print(tdf)

                                                Review  Score  label
0    전주가면 무조건 가봐야하는곳한옥하나하나 너무 멋지고 잘 꾸며져 있어요그러나 길거리 ...    4.0      1
1    젊음의 거리, 사진찍는 거리입니다. 저렴하게 빌리는 옷 덕분에 여기 저기 한복 입고...    4.0      1
2    한복입고 사진찍긴 좋아요거리음식거리라서 먼가 특별하다 싶지 않아요물론 처음가서 보기...    3.0      1
3    전주여행을 생각한다면 꼭 한번 가게되는 곳이라고 생각합니다.한옥을 보는 재미도 있지...    4.0      1
4    첫째날은 사람구경둘째날은 먹거리 먹고 꽃구경 셋째날은 세세한 곳 소리관 김치   그...    5.0      1
..                                                 ...    ...    ...
502  버스터미널에서 5분거리에 위치해있어 전북대학교를 방문하는 길에 잠시 스쳐지나갔던 곳...    2.0      0
636  시설면에나 운영면에서 아쉬운접이 많았습니다.조금더 시의 지원을 받았으면 합니다.의자...    2.0      0
680  주말인데도 가게들이 드문드문 문을 열고 영업하며, 활력이 전혀 없어 이미 쇠락한 느...    2.0      0
684  알쓸신잡 보고 기대하고 갔어요. 일단 아기자기하게 이쁜 가게들이 많아보였지만 평일이...    2.0      0
691  침대방 들어갈 수 있는 사람은 팔목에 띠를 착용 한 사람만 들어 갈 수 있음1번 침...    1.0      0

[759 rows x 3 columns]


In [ ]:
tdf_l=list(np.array(tdf.label.tolist()))
print(tdf_l[:10])

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]


In [ ]:
ss=[]
for a in s:
  if (a>0):
    ss.append(1)
  else:
    ss.append(0)

In [ ]:
c=0
for i in range(len(tdf_l)):
  if (tdf_l[i]==ss[i]):
    c+=1

print(c)
print(len(tdf_l))
print(c/len(tdf_l)*100)

416
759
54.80895915678524
